In [1]:
## All to folder
## generate prompt
# !ls data
# import time
# time.sleep(60*30)

# Try to do:
# torch.cuda.empty_cache()

In [2]:
import os
import json

import transformers
from peft import PeftModel
from transformers import LlamaForCausalLM as LLaMAForCausalLM
from transformers import LlamaTokenizer as LLaMATokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model
from datasets import load_dataset
from EvaluateTestSet import EvaluateTestSet
from transformers.integrations import TensorBoardCallback
from transformers import GenerationConfig

def init_lora_model_and_tokenizer(default_model,
                             LORA_R,
                             LORA_ALPHA,
                             LORA_DROPOUT
                            ):


    """
        
    """
    model = LLaMAForCausalLM.from_pretrained(
    default_model,
    load_in_8bit=True,
    device_map="auto",
    )
    tokenizer = LLaMATokenizer.from_pretrained(
        default_model, add_eos_token=True
    )

    model = prepare_model_for_int8_training(model)

    config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
    )

    model = get_peft_model(model, config)

    tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token

    return model, tokenizer



class MyCustomCallback(TensorBoardCallback):
    #log_bleu_steps_factor = 5
    bleu_generation_max_new_tokens = 30
    bleu_fn_test_data = "temp/t2c_answers.json"
    bleu_fn_etalon = "temp/answers.json"
    log_step = 0
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        super().on_log(args, state, control, logs=logs, **kwargs)
        #print("kwargs", len(kwargs), kwargs.keys())
        if self.tb_writer is not None:
            #print(state)
            #print(state.global_step)
            #print(self.log_step)
            if (self.log_step % self.log_bleu_steps_factor ==0):
                model = kwargs['model']
                tokenizer = kwargs['tokenizer']
                
                model.eval()
                assert not model.training
                generation_config = GenerationConfig(max_new_tokens = self.bleu_generation_max_new_tokens,
                                                     # min_new_tokens = 5,
                                                     temperature = 1.0
                                                    )
                print("generation_config:", generation_config)
                evaluator = EvaluateTestSet(generation_config = generation_config,
                                            fn_test_data = self.bleu_fn_test_data,
                                            fn_etalon = self.bleu_fn_etalon,
                                            batch_size = 1
                                       )

                metric_res = evaluator.evaluate(model=model, 
                                                tokenizer=tokenizer,
                                               )
                model.train()
                assert model.training
                print(metric_res)
                for key, val in metric_res.items():
                    #add "custom/something"
                    self.tb_writer.add_scalar(key, val, state.global_step)
                self.tb_writer.flush()
            self.log_step += 1


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


In [3]:
# !pip list
# !pip install bitsandbytes==0.37.2


In [4]:
CONFIG_PATH = "/root/experiments_configs/"
EXPERIMENTS_PATH = "/root/experiments/"
experiment_name = "t2c_concode_220428_v31"
# t2c_concode_220428_v18.json

In [5]:
!ls /root/experiments_configs

t2c_concode_220428_v14_config.json  t2c_concode_220428_v20_config.json
t2c_concode_220428_v15_config.json  t2c_concode_220428_v22_config.json
t2c_concode_220428_v16_config.json  t2c_concode_220428_v30_config.json
t2c_concode_220428_v18_config.json  t2c_concode_220428_v31.json
t2c_concode_220428_v19_config.json  t2c_concode_220428_v31_config.json


In [6]:
# experiment_config

In [7]:
current_config_path = os.path.join(CONFIG_PATH, experiment_name + "_config.json")
experiment_config = json.load(open(current_config_path, "r"))

assert experiment_config['experiment_name'] == experiment_name

In [8]:
experiment_config['resume_from_checkpoint'] = False

In [9]:
assert experiment_config['resume_from_checkpoint'] == True

AssertionError: 

In [10]:
assert experiment_config['experiment_name'] == experiment_name

In [11]:
current_experiment_path = os.path.join(EXPERIMENTS_PATH, experiment_name)

In [12]:
!mkdir {current_experiment_path}

In [13]:
json.dump(experiment_config, open(current_experiment_path + \
                                  "/experiment_config.json", 
                                  "w+"
                                 )
         )

In [14]:
setattr(MyCustomCallback, "log_bleu_steps_factor", experiment_config['log_bleu_steps_factor'])

In [15]:
MyCustomCallback.log_bleu_steps_factor

50

In [16]:
model, tokenizer = init_lora_model_and_tokenizer(default_model = experiment_config["default_model"],
                                                 LORA_R = experiment_config["LORA_R"],
                                                 LORA_ALPHA = experiment_config["LORA_ALPHA"],
                                                 LORA_DROPOUT = experiment_config["LORA_DROPOUT"]
                                                )


data = load_dataset("json", 
                    data_files = {"train": experiment_config["fn_train_dataset"],
                                  "eval":  experiment_config["fn_eval_dataset"]
                                 }
                   )

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3ac2744fedc77f2f/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
tokenizer_val = LLaMATokenizer.from_pretrained(
    experiment_config['default_model'], add_eos_token=True
)
tokenizer_val.pad_token_id = 0  # unk. we want this to be different from the eos token


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [18]:
# experiment_config["logging_steps"] = 1
# experiment_config["eval_steps"] = 1

In [19]:
experiment_config
# {'experiment_name': 't2c_concode_220428_v19',
#  'fn_train_dataset': '/root/data/t2c_train.json',
#  'fn_eval_dataset': '/root/data/t2c_answers.json',
#  'default_model': 'decapoda-research/llama-7b-hf',
#  'MICRO_BATCH_SIZE': 2,
#  'BATCH_SIZE': 10,
#  'EPOCHS': 2,
#  'LEARNING_RATE': 0.0002,
#  'CUTOFF_LEN': 256,
#  'LORA_R': 16,
#  'LORA_ALPHA': 16,
#  'LORA_DROPOUT': 0.05,
#  'warmup_steps': 200,
#  'fp16': True,
#  'logging_steps': 10,
#  'eval_steps': 100,
#  'evaluation_strategy': 'steps',
#  'save_total_limit': 1,
#  'save_strategy': 'steps',
#  'save_steps': 500,
#  'seed': 42,
#  'logging_strategy': 'steps',
#  'report_to': 'tensorboard',
#  'mlm': False,
#  'truncation': True,
#  'padding': 'max_length',
#  'config_use_cache': False,
#  'resume_from_checkpoint': False,
#  'bleu_batch_size': 5,
#  'GRADIENT_ACCUMULATION_STEPS': 5,
#  'log_bleu_steps_factor': 50}

{'experiment_name': 't2c_concode_220428_v31',
 'fn_train_dataset': '/root/data/t2c_train.json',
 'fn_eval_dataset': '/root/data/t2c_answers.json',
 'default_model': 'decapoda-research/llama-7b-hf',
 'MICRO_BATCH_SIZE': 2,
 'BATCH_SIZE': 10,
 'EPOCHS': 2,
 'LEARNING_RATE': 0.0002,
 'CUTOFF_LEN': 256,
 'LORA_R': 1024,
 'LORA_ALPHA': 1024,
 'LORA_DROPOUT': 0.05,
 'warmup_steps': 200,
 'fp16': True,
 'logging_steps': 10,
 'eval_steps': 100,
 'evaluation_strategy': 'steps',
 'save_total_limit': 1,
 'save_strategy': 'steps',
 'save_steps': 100,
 'seed': 42,
 'logging_strategy': 'steps',
 'report_to': 'tensorboard',
 'mlm': False,
 'truncation': True,
 'padding': 'max_length',
 'config_use_cache': False,
 'resume_from_checkpoint': False,
 'bleu_batch_size': 5,
 'GRADIENT_ACCUMULATION_STEPS': 5,
 'log_bleu_steps_factor': 50,
 'load_best_model_at_end': False}

In [20]:
# experiment_config["resume_from_checkpoint"]

In [21]:
from prompter import Prompter
prompter = Prompter()

def generate_prompt(data_point):
    if "input" in data_point and data_point["input"]:
        return prompter.generate_prompt(instruction = data_point["instruction"],
                                        input = data_point["input"],
                                        label = data_point["output"]
                                       )
    else:
        return prompter.generate_prompt(instruction = data_point["instruction"],
                                        #input = None,
                                        label = data_point["output"]
                                       )

/root/ipynb/prompter/templates/


In [22]:
# experiment_config["resume_from_checkpoint"] =

In [ ]:
# def generate_prompt(data_point):
#     # sorry about the formatting disaster gotta move fast
#     if data_point["input"]:
#         return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
# ### Instruction:
# {data_point["instruction"]}
# ### Input:
# {data_point["input"]}
# ### Response:
# {data_point["output"]}"""
#     else:
#         return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
# ### Instruction:
# {data_point["instruction"]}
# ### Response:
# {data_point["output"]}"""


data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=experiment_config["truncation"],
        max_length=experiment_config["CUTOFF_LEN"],
        padding=experiment_config["padding"]
    )
)

trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer_val,
    train_dataset=data["train"],
    eval_dataset=data['eval'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=experiment_config["MICRO_BATCH_SIZE"],
        gradient_accumulation_steps=experiment_config["GRADIENT_ACCUMULATION_STEPS"],
        warmup_steps=experiment_config["warmup_steps"],
        num_train_epochs=experiment_config["EPOCHS"],
        learning_rate=experiment_config["LEARNING_RATE"],
        fp16=experiment_config["fp16"],
        logging_steps=experiment_config["logging_steps"],        
        evaluation_strategy = experiment_config['evaluation_strategy'],
        eval_steps=experiment_config["eval_steps"],
        output_dir=current_experiment_path,#"lora-alpaca",
        save_total_limit=experiment_config["save_total_limit"],
        save_strategy = experiment_config["save_strategy"],
        
        save_steps = experiment_config["save_steps"],
        seed=experiment_config["seed"],
        logging_dir=current_experiment_path,
        logging_strategy=experiment_config["logging_strategy"],
        report_to=experiment_config["report_to"],
        load_best_model_at_end = experiment_config["load_best_model_at_end"]
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, 
                                                               mlm=experiment_config["mlm"]
                                                              ),
    callbacks = [MyCustomCallback]
)
model.config.use_cache = experiment_config["config_use_cache"]
# print(len(trainer.optimizer.state['found_inf_per_device']))


trainer.train(resume_from_checkpoint=experiment_config["resume_from_checkpoint"])

model.save_pretrained(current_experiment_path)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
100,1.045600,1.145902
200,0.998100,1.136443
300,0.922300,1.142087
400,0.994800,1.139181
500,0.940100,1.141253
600,1.019800,1.144103
700,0.919600,1.136041
800,0.973900,1.146860
900,0.976600,1.144602
1000,0.974800,1.150033


generation_config: GenerationConfig {
  "max_new_tokens": 30,
  "transformers_version": "4.28.1"
}



...

100%|████████████████████████████████████████| 30/30 [00:00<00:00, 53934.47it/s]

{'EM': 0.0, 'BLEU': 0.2009365018689266, 'brevity_penalty': 0.6385332304644112, 'ratio': 0.6903304773561811, 'translation_length': 564, 'reference_length': 817, 'precisions_0': 0.5876106194690266, 'precisions_1': 0.3775700934579439, 'precisions_2': 0.2509881422924901, 'precisions_3': 0.1761006289308176}


generation_config: GenerationConfig {
  "max_new_tokens": 30,
  "transformers_version": "4.28.1"
}



100%|███████████████████████████████████████| 30/30 [00:00<00:00, 180013.05it/s]


{'EM': 0.0, 'BLEU': 0.21060673693233478, 'brevity_penalty': 0.7523217443777271, 'ratio': 0.7784577723378213, 'translation_length': 636, 'reference_length': 817, 'precisions_0': 0.5400313971742543, 'precisions_1': 0.34596375617792424, 'precisions_2': 0.21663778162911612, 'precisions_3': 0.15173674588665448}


In [25]:
current_experiment_path

'/root/experiments/t2c_concode_220428_v31'

In [24]:
!ls {current_experiment_path}

1684072293.7154148   checkpoint-20000
1684072293.727496    events.out.tfevents.1684072293.8d048d63ed1a.18071.0
adapter_config.json  events.out.tfevents.1684072293.8d048d63ed1a.18071.2
adapter_model.bin    experiment_config.json


In [1]:
0

0

In [ ]:
!ls -lah {current_experiment_path}

In [22]:
model.save_pretrained(current_experiment_path)

In [28]:
experiment_config

{'experiment_name': 't2c_concode_220428_v22',
 'fn_train_dataset': '/root/data/t2c_train.json',
 'fn_eval_dataset': '/root/data/t2c_answers.json',
 'default_model': 'decapoda-research/llama-7b-hf',
 'MICRO_BATCH_SIZE': 2,
 'BATCH_SIZE': 10,
 'EPOCHS': 2,
 'LEARNING_RATE': 0.0002,
 'CUTOFF_LEN': 256,
 'LORA_R': 64,
 'LORA_ALPHA': 64,
 'LORA_DROPOUT': 0.05,
 'warmup_steps': 200,
 'fp16': True,
 'logging_steps': 10,
 'eval_steps': 100,
 'evaluation_strategy': 'steps',
 'save_total_limit': 1,
 'save_strategy': 'steps',
 'save_steps': 500,
 'seed': 42,
 'logging_strategy': 'steps',
 'report_to': 'tensorboard',
 'mlm': False,
 'truncation': True,
 'padding': 'max_length',
 'config_use_cache': False,
 'resume_from_checkpoint': False,
 'bleu_batch_size': 5,
 'GRADIENT_ACCUMULATION_STEPS': 5,
 'log_bleu_steps_factor': 50}

In [27]:
model.save_pretrained(current_experiment_path)

In [25]:
!rm -rf {current_experiment_path}

In [30]:
!ls /root/experiments

t2c_concode_220428_v12	t2c_concode_220428_v16	t2c_concode_220428_v22
t2c_concode_220428_v13	t2c_concode_220428_v18	t2c_concode_220428_v30
t2c_concode_220428_v14	t2c_concode_220428_v19	t2c_concode_220428_v9
t2c_concode_220428_v15	t2c_concode_220428_v20


In [62]:
# !du -hs /root/ | sort -h

14G	/root/


In [58]:
# !du -lahS /root/
# !du -hs /root/.cache/huggingface/* | sort -h

13G	/root/.cache/huggingface/hub


In [60]:
# !ls /root/.cache/huggingface/hub

In [61]:
# !ls /root/experiments/

In [54]:
!df -h .

Filesystem      Size  Used Avail Use% Mounted on
overlay          45G   17G   29G  37% /


In [ ]:
!ls -lah {current_experiment_path}

In [ ]:
current_experiment_path

In [ ]:
!ls -lah /root/experiments/t2c_concode_220428_v20/checkpoint-500

In [ ]:
!ls -lah /root/experiments/t2c_concode_220428_v20/checkpoint-500

In [ ]:
# !rm -rf /root/experiments/t2c_concode_220428_v19/checkpoint-20000/
# !df -h .

In [ ]:
# !rm -rf {current_experiment_path}/checkpoint-1000

In [ ]:
print(123)

In [ ]:
# import sys
# sys.exit()

In [25]:
!df -h .

Filesystem      Size  Used Avail Use% Mounted on
overlay          45G   45G  3.6M 100% /


In [47]:
!ls /root/experiments/t2c_concode_220428_v13/

1683303090.6023178   adapter_model.bin
1683303090.6101534   events.out.tfevents.1683303090.8d048d63ed1a.5752.0
1683303146.4560587   events.out.tfevents.1683303090.8d048d63ed1a.5752.2
1683303146.4694548   events.out.tfevents.1683303146.8d048d63ed1a.5752.4
1683303388.633225    events.out.tfevents.1683303146.8d048d63ed1a.5752.6
1683303388.645383    events.out.tfevents.1683303388.8d048d63ed1a.5752.10
1683303483.1163416   events.out.tfevents.1683303388.8d048d63ed1a.5752.8
1683303483.1291792   events.out.tfevents.1683303483.8d048d63ed1a.5752.12
1683303652.147785    events.out.tfevents.1683303483.8d048d63ed1a.5752.14
1683303652.1612782   events.out.tfevents.1683303652.8d048d63ed1a.5752.16
1683303676.8182936   events.out.tfevents.1683303652.8d048d63ed1a.5752.18
1683303676.8320808   events.out.tfevents.1683303676.8d048d63ed1a.5752.20
1683303700.7354565   events.out.tfevents.1683303676.8d048d63ed1a.5752.22
1683303700.7492273   events.out.tfevents.1683303700.8d048d63ed1a.5752.24
1683303738.197177

In [29]:
!df -h .

Filesystem      Size  Used Avail Use% Mounted on
overlay          45G   24G   22G  54% /


In [45]:
!df -h .

Filesystem      Size  Used Avail Use% Mounted on
overlay          45G   24G   22G  54% /
